<a href="https://colab.research.google.com/github/mssnowy/c115/blob/main/C115_Boilerplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load the Dataset

In [1]:
!git clone https://github.com/procodingclass/PRO-C114-Text-Sentiment-Dataset

Cloning into 'PRO-C114-Text-Sentiment-Dataset'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (20/20), 2.94 MiB | 2.08 MiB/s, done.


## Pandas:
**Pandas** is an open-source library built on top of Numpy and Matplotlib. It provides high-performance, easy-to-use data structures and data analysis tools.

A DataFrame is a two-dimensional data structure, i.e., data is aligned in a tabular fashion in rows and columns. Pandas DataFrame consists of three principal components, the data, rows, and columns.

Pandas DataFrame will be created by loading the datasets from existing MS Excel files, CSV files or SQL Database. Pandas DataFrame can also be created from the lists, dictionaries etc.



In [2]:
#Import pandas
import pandas as pd
#read excel file
train_data_raw=pd.read_excel("/content/PRO-C114-Text-Sentiment-Dataset/text-emotion-training-dataset.xlsx")
#display first five entries of training dataset
train_data_raw.head(10)

,Text_Emotion
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger
5,ive been feeling a little burdened lately wasn...
6,ive been taking or milligrams or times recomme...
7,i feel as confused about life as a teenager or...
8,i have been with petronas for years i feel tha...
9,i feel romantic too;love


## Split the rows in two columns as Text and Emotions



In [3]:
#Use split() method to separate the Text and Emotions
train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(),columns=['Text','Emotion'])
train_data.head()

<ipython-input-3-f40b6f5a6685>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(),columns=['Text','Emotion'])


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


## Giving labels to Emotions

In [4]:
#Find unique emotions
train_data['Emotion'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [5]:
#Create a Dictionary to replace emotions with labels
encode_emotions={'anger':0,'fear':1, 'joy':2,'love':3,'sadness':4,'surprise':5}
train_data.replace(encode_emotions,inplace=True)
train_data.head()

,Text,Emotion
0,i didnt feel humiliated,4
1,i can go from feeling so hopeless to so damned...,4
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0


## Convert Dataframe to list of dataset

In [6]:
# Define two list for sentences and emotions
training_sentences=[]
training_labels=[]
# append text and emotions in the list using the 'loc' method
for i in range (len(train_data)):
  sentence=train_data.loc[i,'Text']
  training_sentences.append(sentence)
  label=train_data.loc[i,'Emotion']
  training_labels.append(label)
#Check a random text and label of the list
training_sentences[50],training_labels[50]

('i need to feel the dough to make sure its just perfect', 2)

## Tokenization & Padding
The act of converting text into numbers is known as **Tokenization**. The Tokenizer class of Keras is used for encoding text input into integer sequence.

**Padding** is important to make all the sentences contain the same number of words. Zero is used for padding the tokenized sequence to make text contain the same number of tokens.

In [7]:
#import Tokenizer and pad_sequences from tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Define parameters for Tokenizer
vocab_size=10000
embedding_dim=16
oov_tok="<OOV>"
training_size=20000
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
#Create a word_index dictionary
word_index=tokenizer.word_index
#Check the tokenized sequence
word_index['the']

6

In [8]:
#import pad_sequences from tensorflow
training_sequences=tokenizer.texts_to_sequences(training_sentences)
print(training_sequences[0])
print(training_sequences[1])
print(training_sequences[2])
#Define parameters for pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding_type='post'
max_length=100
trunc_type='post'
#Check the padded sequence
training_padded=pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
training_padded

[2, 139, 3, 679]
[2, 40, 101, 60, 8, 15, 494, 5, 15, 3496, 553, 32, 60, 61, 128, 148, 76, 1480, 4, 22, 1255]
[17, 3060, 7, 1149, 5, 286, 2, 3, 495, 438]


array([[   2,  139,    3, ...,    0,    0,    0],
       [   2,   40,  101, ...,    0,    0,    0],
       [  17, 3060,    7, ...,    0,    0,    0],
       ...,
       [   2,    3,  327, ...,    0,    0,    0],
       [   2,    3,   14, ...,    0,    0,    0],
       [   2,   47,    7, ...,    0,    0,    0]], dtype=int32)

In [10]:
#c115
import numpy as np
training_padded=np.array(training_padded)
print(training_padded)
training_labels=np.array(training_labels)
print(training_labels)

[[   2  139    3 ...    0    0    0]
 [   2   40  101 ...    0    0    0]
 [  17 3060    7 ...    0    0    0]
 ...
 [   2    3  327 ...    0    0    0]
 [   2    3   14 ...    0    0    0]
 [   2   47    7 ...    0    0    0]]
[4 4 0 ... 2 0 4]


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.layers import Conv1D,Dropout,MaxPooling1D

model=tf.keras.Sequential([
    Embedding(vocab_size,embedding_dim,input_length=max_length),
    Dropout(0.2),

    Conv1D(filters=256,kernel_size=3,activation="relu"),
    MaxPooling1D(pool_size=3),

    Conv1D(filters=128,kernel_size=3,activation="relu"),
    MaxPooling1D(pool_size=3),

    LSTM(128),
    Dense(128,activation="relu"),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dense(6,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 256)           12544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 32, 256)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 128)           98432     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 10, 128)          0         
 1D)                                                    

In [19]:
num_epochs=5
history=model.fit(training_padded,training_labels,epochs=num_epochs,verbose=2)

Epoch 1/5
500/500 - 30s - loss: 1.1844 - accuracy: 0.4324 - 30s/epoch - 60ms/step
Epoch 2/5
500/500 - 39s - loss: 0.6027 - accuracy: 0.7824 - 39s/epoch - 78ms/step
Epoch 3/5
500/500 - 35s - loss: 0.4017 - accuracy: 0.8705 - 35s/epoch - 70ms/step
Epoch 4/5
500/500 - 34s - loss: 0.3243 - accuracy: 0.8931 - 34s/epoch - 67ms/step
Epoch 5/5
500/500 - 33s - loss: 0.2540 - accuracy: 0.9163 - 33s/epoch - 67ms/step


In [20]:
model.save('Text_Emotion.h5')

In [21]:
sentence=['i am happy to meet my friends. we are planning to go to a party.',
          "i had a bad day at school. i got hurt while playing football."]
sequences=tokenizer.texts_to_sequences(sentence)

padded=pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
result=model.predict(padded)
predict_class=np.argmax(result,axis=1)
predict_class

1/1 [==============================] - 1s 610ms/step


array([2, 4])